In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#Crated  by: Luisa Fernanda Buriticá Ruíz
#Date: December 16, 2024
#Contact: fernanda.buritica@udea.edu.co
#Title: Automatic Data Download from IDEAM V3

In [2]:
# 1. LIBRARIES
# 1.1 Imported libraries (6)
import os               # System interaction, file manipulation
import pandas as pd     # Tabular data manipulation and analysis
import time             # Time-related functions
!pip install sodapy
import config

# 1.2 Imported modules (4)
from requests.exceptions import ReadTimeout, ConnectionError, HTTPError, ChunkedEncodingError
from collections import Counter
from datetime import datetime, timedelta  # Advanced date and time manipulation
from sodapy import Socrata                # Interacting with Socrata API datasets

In [3]:
# 2. FUNCIONES
#Function for handling IDEAM's CSV files of stations, both from the IDEAM entity and other entities.
def CrearCarpetas(nombres_carpetas, ruta):
    for nombre in nombres_carpetas:
        ruta_completa = os.path.join(ruta, nombre)
        if not os.path.exists(ruta_completa):
            os.mkdir(ruta_completa)
def get_most_common_date_from_files(folder_path):
    all_dates = []

    # Verificar si la carpeta está vacía
    if not os.listdir(folder_path):
        # Si la carpeta está vacía, retornar una fecha predeterminada
        return datetime(1999, 1, 1)
    
    # Iterar sobre los archivos en la carpeta
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.isfile(file_path) and file_name.endswith('.csv'):
            # Leer el archivo CSV
            df = pd.read_csv(file_path)

            # Asegurarse de que la columna 'fecha' está en formato datetime
            df['fecha'] = pd.to_datetime(df['fecha'])

            # Obtener la última fila y su fecha
            last_date = df['fecha'].iloc[-1]
            all_dates.append(last_date)

    # Calcular la fecha promedio más repetida
    if all_dates:
        most_common_date = Counter(all_dates).most_common(1)[0][0]
        return most_common_date
    else:
        # Si no se encuentra ninguna fecha, retornar la fecha predeterminada
        return datetime(1999, 1, 1)
def get_max_date_from_files(folder_path):
    all_dates = []

    # Verificar si la carpeta está vacía
    if not os.listdir(folder_path):
        # Si la carpeta está vacía, retornar una fecha predeterminada
        return datetime(1999, 1, 1)
    
    # Iterar sobre los archivos en la carpeta
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.isfile(file_path) and file_name.endswith('.csv'):
            # Leer el archivo CSV
            df = pd.read_csv(file_path)

            # Asegurarse de que la columna 'fecha' está en formato datetime
            df['fecha'] = pd.to_datetime(df['fecha'])

            # Obtener la última fila y su fecha
            last_date = df['fecha'].iloc[-1]
            all_dates.append(last_date)

    # Calcular la fecha máxima
    if all_dates:
        max_date = max(all_dates)  # Encuentra la fecha máxima
        return max_date
    else:
        # Si no se encuentra ninguna fecha, retornar la fecha predeterminada
        return datetime(1999, 1, 1)


In [4]:
# 3. INPUT INFORMATION
# 3.1 Paths
path_in1=config.path_in1_AUT
# 3.2 Databases and connection parameters
client = config.client
# 3.3 Vectors
# 0: Datasets that already have more than 95% loaded into the Alejandria database.
dicc1_variables = config.dicc1_variables
# 3.4 Inicial configuration
ColumnsQueryOBS=['codigoestacion,fechaobservacion,valorobservado']

In [5]:
# 4. PROCESSES
# 4.2 Step PP 2. For loop, Variables.
# Get the current date
current_date = datetime.now()
for key, value in dicc1_variables.items():
    print(f'Beginning the analysis of the variable: {value[2]}')  # Print the current analysis being processed
    # Create folders
    CrearCarpetas([value[2]], f'{path_in1}')
    
    # Initial configuration
    if value[4]==0:start_date = get_most_common_date_from_files(f'{path_in1}/{value[2]}')
    if value[4]==1:start_date =get_max_date_from_files(f'{path_in1}/{value[2]}')
    limit_columns = 500000
    days_to_add = 5  # Initially add 5 days
    hours_to_add = 24  # Variable to subtract hours when days are 1

    # Loop while end_date is less than or equal to the current date
    while start_date <= current_date + timedelta(days=5):
        # Format start_date and end_date
        start_date_str = start_date.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]
        end_date = start_date + timedelta(days=days_to_add)
        end_date_str = end_date.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]
        
        # Get the data
        consulta = {
            "select": "*",  # Get all information
        }
        while True:
            QDate = f"((fechaobservacion >= '{start_date_str}') AND (fechaobservacion <= '{end_date_str}'))"
            try:
                df1 = pd.DataFrame(client.get(value[0], where=QDate, select=ColumnsQueryOBS, limit=limit_columns))
                print(f"Retrieved {len(df1)} records for {start_date_str} to {end_date_str}")
                
                # If the number of records equals the limit, reduce the days and retry the query
                if len(df1) == limit_columns:
                    if days_to_add > 1:
                        # If we can still reduce the days, reduce the days by 1 (minimum 1 day)
                        days_to_add = max(1, days_to_add - 1)
                        # If no more days can be reduced, start subtracting hours
                        end_date = start_date + timedelta(days=days_to_add)
                        end_date_str = end_date.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]
                        
                    else:
                        # Start subtracting hours
                        hours_to_add = max(1, hours_to_add - 1)
                        end_date = start_date + timedelta(hours=hours_to_add)
                        end_date_str = end_date.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]
                    continue  # Retry the same time window with fewer days or hours
                if len(df1)==0: break
                # Processing of dataframe
                grouped = df1.groupby('codigoestacion')
                
                for station_code, group in grouped:
                    # Prepare the dataframe with date and observed value columns
                    df2 = group[['fechaobservacion', 'valorobservado']].reset_index(drop=True)
                    df2.columns = ['fecha', 'valor']
                    
                    # Convert 'fechaobservacion' to datetime
                    df2['fecha'] = pd.to_datetime(df2['fecha'])
                    
                    # Define the full path where the file will be saved
                    file_path = f'{path_in1}/{value[2]}/{station_code}.csv'
                    
                    # Check if the file already exists
                    if os.path.exists(file_path):
                        # Load the existing file
                        existing_df = pd.read_csv(file_path)
                        
                        # Convert the 'fechaobservacion' column from the existing file to datetime (if it's not already)
                        existing_df['fecha'] = pd.to_datetime(existing_df['fecha'])
                        
                        # Merge, removing duplicate dates and keeping the first one
                        df2 = pd.concat([existing_df, df2]).drop_duplicates(subset=['fecha'], keep='first').sort_values('fecha')
                        df2.reset_index(drop=True, inplace=True)  # Reset the index after the merge
                    
                    # Sort the dates from most recent to least before saving
                    df2 = df2.sort_values('fecha', ascending=False)
                    #df2 = df2.sort_values(by='fecha', ascending=False)
                    df2.to_csv(file_path, index=False, encoding='utf-8-sig')  # Save the updated file (new or merged)
                break  # Exit the loop when valid data is retrieved (less than the limit_columns)
            except (ReadTimeout, ConnectionError, HTTPError, ChunkedEncodingError) as e:
                print(f"\n ---> {type(e).__name__}: {str(e)}. Waiting 20 seconds...")
                time.sleep(20)  # Esperar 20 segundos antes de reintentar
            #except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError, requests.exceptions.HTTPError) as e:
            #    print(f"\n ---> {type(e).__name__}: {str(e)}. Waiting 20 seconds...")
            #    
            #    time.sleep(20)  # Wait 20 seconds before retrying
        # Update start_date for the next iteration (move to the next time window)
        start_date = end_date

Beginning the analysis of the variable: wind_direction
Retrieved 0 records for 1999-01-01T00:00:00.000 to 1999-01-06T00:00:00.000
Retrieved 0 records for 1999-01-06T00:00:00.000 to 1999-01-11T00:00:00.000
Retrieved 0 records for 1999-01-11T00:00:00.000 to 1999-01-16T00:00:00.000

 ---> HTTPError: 500 Server Error: Server Error.
	Internal error: please include code 48a2d366-5f57-46e8-815a-fe66d4929c83 if you report the error. Waiting 20 seconds...


KeyboardInterrupt: 